In [18]:
import pandas as pd
import numpy as np

# 시도명 매핑 딕셔너리 (변동 없음)
sido_map = {
    '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
    '광주': '광주광역시', '울산': '울산광역시', '인천': '인천광역시',
    '대전': '대전광역시',
    
    # 도 단위 및 기타 지역 매핑
    '북춘천': '강원도', '북강릉': '강원도', '울릉도': '경상북도',
    '안동': '경상북도', '포항': '경상북도', '창원': '경상남도',
    '수원': '경기도', '청주': '충청북도', '전주': '전라북도',
    '목포': '전라남도', '여수': '전라남도', '흑산도': '전라남도',
    '홍성': '충청남도', '백령도': '인천광역시', '서귀포': '제주도',
    '제주': '제주도',
    
    # 기타 추가
    '속초': '강원도', '철원': '강원도', '동두천': '경기도', '파주': '경기도',
    '대관령': '강원도', '춘천': '강원도', '강릉': '강원도', '동해': '강원도',
    '원주': '강원도', '영월': '강원도', '충주': '충청북도', '서산': '충청남도',
    '울진': '경상북도', '추풍령': '경상북도', '상주': '경상북도',
    '군산': '전라북도', '통영': '경상남도', '완도': '전라남도', '고창': '전라북도',
    '순천': '전라남도', '진도(첨찰산)': '전라남도', '서청주': '충청북도',
    '고산': '제주도', '성산': '제주도', '진주': '경상남도',
    '강화': '인천광역시', '양평': '경기도', '이천': '경기도',
    '인제': '강원도', '홍천': '강원도', '태백': '강원도', '정선군': '강원도',
    '제천': '충청북도', '보은': '충청북도', '천안': '충청남도',
    '보령': '충청남도', '부여': '충청남도', '금산': '충청남도',
    '세종': '세종특별자치시', '부안': '전라북도', '임실': '전라북도',
    '정읍': '전라북도', '남원': '전라북도', '장수': '전라북도',
    '고창군': '전라북도', '영광군': '전라남도', '김해시': '경상남도',
    '순창군': '전라북도', '북창원': '경상남도', '양산시': '경상남도',
    '보성군': '전라남도', '강진군': '전라남도', '장흥': '전라남도',
    '해남': '전라남도', '고흥': '전라남도', '의령군': '경상남도',
    '함양군': '경상남도', '광양시': '전라남도', '진도군': '전라남도',
    '봉화': '경상북도', '영주': '경상북도', '문경': '경상북도',
    '청송군': '경상북도', '영덕': '경상북도', '의성': '경상북도',
    '구미': '경상북도', '영천': '경상북도', '경주시': '경상북도',
    '거창': '경상남도', '합천': '경상남도', '밀양': '경상남도',
    '산청': '경상남도', '거제': '경상남도', '남해': '경상남도',
    '북부산': '부산광역시'
}


# 1. 데이터 로드 (파일 경로 수정 필요)
file_path = '일별_기온_강수량_일조15~24.csv'
try:
    df_precipitation = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    # 실제 환경에서는 exit() 대신 에러 처리를 수행하거나 테스트 데이터를 로드합니다.
    # 여기서는 코드만 수정하여 제공합니다.
    pass


# 2. 날짜 형식 변환 및 year 컬럼 추가
df_precipitation['일시'] = pd.to_datetime(df_precipitation['일시'], format='%Y-%m-%d', errors='coerce')
df_precipitation['year'] = df_precipitation['일시'].dt.year


# 3. 시도명 매핑
df_precipitation['시도명'] = df_precipitation['지점명'].map(sido_map)
df_precipitation.drop('지점명', axis=1, inplace=True)


# 4. 장마 기간 데이터 필터링 후 (연, 시도명)으로 평균 집계 (6월 15일~7월 31일)
start_month_day = '06-15'
end_month_day = '07-31'

df_precipitation_jangma = df_precipitation[
    df_precipitation['일시'].apply(
        # 날짜 비교를 위해 연도를 2000년으로 통일
        lambda x: pd.to_datetime(f"{x.year}-{start_month_day}") <= x <= pd.to_datetime(f"{x.year}-{end_month_day}")
    )
]

# 평균 일강수량 계산
jangma_avg = df_precipitation_jangma.groupby(['year', '시도명'], as_index=False)['일강수량(mm)'].mean().round(3)
jangma_avg.rename(columns={'일강수량(mm)': '장마_평균_일강수량(mm)'}, inplace=True)


# 5. 세종특별자치시 데이터 보완 및 중복 제거

sejong_to_add = []
years_to_check = [2015, 2016, 2017, 2018]

# 2015, 2016, 2017년 세종 데이터 대체 (대전광역시 데이터 사용)
for year in [2015, 2016, 2017]:
    # 해당 연도의 대전 평균 강수량 값 찾기
    daejeon_data = jangma_avg[
        (jangma_avg['year'] == year) & (jangma_avg['시도명'] == '대전광역시')
    ]['장마_평균_일강수량(mm)']
    
    if not daejeon_data.empty:
        # 데이터가 있으면 세종 데이터로 추가
        sejong_data = {
            'year': year,
            '시도명': '세종특별자치시',
            '장마_평균_일강수량(mm)': daejeon_data.values[0]
        }
        sejong_to_add.append(sejong_data)

# 2018년 세종 데이터 처리:
# 1. 2018년 세종 데이터 중복 제거
jangma_avg.drop_duplicates(subset=['year', '시도명'], keep='first', inplace=True)

# 2. 2018년 세종 데이터가 (중복 제거 후에도) 누락되었을 경우 대전 데이터로 대체
is_sejong_2018_exists = '세종특별자치시' in jangma_avg[jangma_avg['year'] == 2018]['시도명'].values

if not is_sejong_2018_exists:
    # 2018년 세종 데이터가 없다면, 대전 데이터로 대체하여 추가
    daejeon_data_2018 = jangma_avg[
        (jangma_avg['year'] == 2018) & (jangma_avg['시도명'] == '대전광역시')
    ]['장마_평균_일강수량(mm)']
    
    if not daejeon_data_2018.empty:
        sejong_data_2018 = {
            'year': 2018,
            '시도명': '세종특별자치시',
            '장마_평균_일강수량(mm)': daejeon_data_2018.values[0]
        }
        sejong_to_add.append(sejong_data_2018)

# 새로운 세종 데이터를 기존 데이터에 추가
if sejong_to_add:
    jangma_avg = pd.concat([jangma_avg, pd.DataFrame(sejong_to_add)], ignore_index=True)


# 6. 최종 정렬 및 결과 저장
jangma_avg = jangma_avg.sort_values(['year', '시도명']).reset_index(drop=True)

output_path = '시도별_장마_평균_일강수량.csv'
jangma_avg.to_csv(output_path, index=False, encoding='utf-8')

print("✅ 시도별 장마 평균 일강수량 데이터 처리가 완료되었습니다.")
print("✅ 2015, 2016, 2017년 세종특별자치시 데이터가 대전광역시 값으로 대체되었습니다.")
print("✅ 2018년 세종특별자치시 데이터 중복이 제거되었습니다.")

✅ 시도별 장마 평균 일강수량 데이터 처리가 완료되었습니다.
✅ 2015, 2016, 2017년 세종특별자치시 데이터가 대전광역시 값으로 대체되었습니다.
✅ 2018년 세종특별자치시 데이터 중복이 제거되었습니다.
